In [1]:

import logging
import os
import sys
import json

import numpy as np
from datasets import load_dataset
import jieba 
from rouge_chinese import Rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import torch

import transformers
from transformers import (
    AutoConfig,
    AutoModel,
    LlamaConfig,
    LlamaTokenizer,
    LlamaForCausalLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainingArguments,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    
    set_seed,
)

os.environ['CUDA_VISIBLE_DEVICES']='0'
os.environ["WANDB_MODE"]='disabled'


from transformers import(
    T5Tokenizer,
    T5ForConditionalGeneration,
    Trainer,
    Seq2SeqTrainer,
)
from peft import LoraConfig, get_peft_model, TaskType
from zero_to_fp32 import load_state_dict_from_zero_checkpoint

/home/lzw/miniconda3/envs/Bert/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from peft import PeftModelForCausalLM

model_name_or_path = '../model/chinese-llama-alpaca-plus-lora-7b'
config = LlamaConfig.from_pretrained(
    model_name_or_path,
    # trust_remote_code=True
)
tokenizer = LlamaTokenizer.from_pretrained(
    model_name_or_path,
    # trust_remote_code=True
)
model = LlamaForCausalLM.from_pretrained(
        model_name_or_path,
        config=config,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.07s/it]


In [ ]:
target_modules = ".*(1[6_9]|2[0-9]|3[0-1]).*(q_proj|k_proj|down_proj|up_proj|gate_proj)"
lora_rank = 8
lora_dropout = 0.1
lora_alpha = 32
print(target_modules)
print(lora_rank)
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=target_modules,
    inference_mode=False,
    r=lora_rank, lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
)
model = get_peft_model(model, peft_config)

In [ ]:
checkpoint = '../checkpoint/PromptCBLUE-alpaca-llama-7b-lora-2e-4/checkpoint-5000'
# checkpoint = '../checkpoint/CHIP-CDEE-2e-4/checkpoint-1000'
model = load_state_dict_from_zero_checkpoint(model, checkpoint).cuda()

In [ ]:
# model.save_pretrained('../model/global_model')
model.print_trainable_parameters()

In [ ]:
your_data_path="../datasets/toy_examples/"
train_file =  os.path.join(your_data_path, 'train.json')
validation_file =  os.path.join(your_data_path, 'dev.json')
test_file =  os.path.join(your_data_path, 'test.json')
# Load dataset
data_files = {}
if train_file is not None:
    data_files["train"] = train_file
    extension = train_file.split(".")[-1]
if validation_file is not None:
    data_files["validation"] = validation_file
    extension = validation_file.split(".")[-1]
if test_file is not None:
    data_files["test"] = test_file
    extension = test_file.split(".")[-1]

lm_datasets = load_dataset(
    extension,
    data_files=data_files,
)


# Get the column names for input/target.
prompt_column = 'input'
response_column = 'target'

column_names = lm_datasets["validation"].column_names
# Temporarily set max_target_length for training.
max_target_length = 196
max_input_length = 256
prefix = ''

def preprocess_function(examples):
    ret = [x + y for x, y in zip(examples[prompt_column], examples[response_column])]
    return tokenizer(ret)

tokenized_dataset = lm_datasets.map(
    preprocess_function,
    batched=True,
    # num_proc=data_args.preprocessing_num_workers,
    num_proc=4,
    remove_columns=column_names,
    load_from_cache_file=False,
)



# Main data processing function that will make each entry its own in the dataset
def single_texts(examples):
    result = examples
    result["labels"] = examples["input_ids"].copy()
    return result


def group_texts(examples):
    # block_size = data_args.max_source_length + data_args.max_target_length
    block_size  = 520
    
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


lm_dataset = tokenized_dataset.map(single_texts, batched=True, num_proc=4)
# lm_dataset = tokenized_dataset
lm_dataset.set_format('torch', columns=['input_ids', 'labels'])

In [ ]:
from torch.utils.data import DataLoader

# dataloader = DataLoader(
#     lm_dataset['test'],
#     batch_size=2,
# )
str = "医疗搜索：我把口香糖吃到肚子里面会不会有什么影响\n回答内容：你好，不会有危险的，口香糖里面的主要成分是蔗糖，同时有食品胶，在体内是不会被吸收的，所以即使孩子吞服，也是不会造成不良影响的出现的。这个情况不必紧张,一般会从大便中拉出来的,以后不要给孩子吃这类食物就可以了.\n上述搜索和回答是否相关？\n选项: 相关，不相关\n答："
input = tokenizer(str, return_tensors='pt', padding='max_length',max_length=400)
output = model(input['input_ids'].cuda())

In [ ]:
tokens = torch.argmax(output.logits[0], dim=-1)
print(tokenizer.decode(tokens))
# print(tokenizer.decode(tokens), '*'*50+'\n',tokenizer.decode(input['labels']))

In [ ]:
generation_config = dict(
    temperature=0.2,
    # top_k=40,
    top_p=0.9,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.3,
    max_new_tokens=400
)
output = model.generate(input['input_ids'], **generation_config)

In [ ]:
model = LlamaForCausalLM.from_pretrained(
    # '../model/ChatMed_llama/'
    '../model/chinese-llama-alpaca-plus-lora-7b/',
    config=config,
).half().cuda()

In [6]:
device='cpu'

str = f"""
### 指令:
根据下文判断上述搜索和回答是否相关？
### 输入:
指令：根据下文判断上述搜索和回答是否相关？回答: 相关,不相关\n医疗搜索：我把口香糖吃到肚子里面会不会有什么影响\n回答内容：你好，不会有危险的，口香糖里面的主要成分是蔗糖，同时有食品胶，在体内是不会被吸收的，所以即使孩子吞服，也是不会造成不良影响的出现的。这个情况不必紧张,一般会从大便中拉出来的,以后不要给孩子吃这类食物就可以了.\n\n\n答：
### 输出:"""


generation_config = dict(
    temperature=0.2,
    # top_k=40,
    top_p=0.9,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.3,
    max_new_tokens=400
)

inputs = tokenizer(str, return_tensors='pt', padding='max_length',max_length=400, )
output = model.generate(
    input_ids=inputs["input_ids"].to(device),
    attention_mask=inputs['attention_mask'].to(device),
    **generation_config
)

/home/lzw/miniconda3/envs/Bert/lib/python3.8/site-packages/transformers/generation/utils.py:1253: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [7]:
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)
# response = output.split("答：\n")[1].strip()
# print(response)


### 指令:
根据下文判断上述搜索和回答是否相关？
### 输入:
指令：根据下文判断上述搜索和回答是否相关？回答: 相关,不相关
医疗搜索：我把口香糖吃到肚子里面会不会有什么影响
回答内容：你好，不会有危险的，口香糖里面的主要成分是蔗糖，同时有食品胶，在体内是不会被吸收的，所以即使孩子吞服，也是不会造成不良影响的出现的。这个情况不必紧张,一般会从大便中拉出来的,以后不要给孩子吃这类食物就可以了.


答：
### 输出:  相关的 。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。  。茬  。茬  。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬    。  。茬
